## Teste 1
##### Programação grosseira em busca dos melhores dados e dos melhores modelos

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from warnings import simplefilter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

plt.rcParams['figure.figsize'] = (25, 8)
#sns.set_style('whitegrid')
sns.set_style('white')
sns.set_context('poster')

In [37]:
df = pd.read_csv('datasets/dataset_cler.csv', sep=";", encoding='utf-16')
del df['Ref']
df.shape

(1445, 115)

In [38]:
df.head(2)

,ampl_9,desv_amp_9,body_9,body_per_9,pavio_sup_9,pavio_sup_per_9,pavio_inf_9,pavio_inf_per_9,high_dist_9,low_dist_9,...,pavio_inf_0,pavio_inf_per_0,high_dist_0,low_dist_0,type_0,atr,qt_bars,oper,take,status
0,70.0,1.17,50.0,71.43,20.0,28.57,0.0,0.0,25.0,35.0,...,5.0,11.11,5.0,5.0,down,58.21,5,Sell,-40.0,Loss
1,80.0,2.67,45.0,56.25,25.0,31.25,10.0,12.5,55.0,5.0,...,5.0,8.33,25.0,15.0,down,59.64,5,Sell,-40.0,Loss


In [39]:
df['status'].value_counts()

Loss    725
Gain    720
Name: status, dtype: int64

In [40]:
df.drop(df.loc[(df['status']!='Loss') & (df['status']!='Gain')].index.values, inplace=True)

In [41]:
df['status'].value_counts()

Loss    725
Gain    720
Name: status, dtype: int64

In [42]:
#train_features = df.columns.difference(['take', 'status', 'oper'])
X = df.drop(['take','status','oper'], axis=1) #df[train_features]
y = df['status']

x_columns = X.columns

In [43]:
index_features_category = []
for index, col in enumerate(X.columns):
    if X.dtypes[index]=='object':
        index_features_category.append(index)
index_features_category[:10]

[10, 21, 32, 43, 54, 65, 76, 87, 98, 109]

In [44]:
encoder = LabelEncoder()
encoder.fit(['inside', 'up', 'down'])
columns_categorical = X.select_dtypes(include=['object']).columns
for col_cat in columns_categorical:
    X[col_cat] = encoder.transform(X[col_cat])

ohe = OneHotEncoder(categorical_features=index_features_category)
X = ohe.fit_transform(X).toarray()

In [45]:
def fit(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    return y_pred 

def fitAll(X_train, X_test, y_train, y_test, algorithms):
    similaridade = []
    for item in algorithms:
        name = item[0]
        algorithm = item[1]
        
        algorithm.fit(X_train, y_train)
        y_pred = algorithm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        #print(name, ' => ', accuracy)
        similaridade.append((accuracy, name))
        
    similaridade.sort()
    similaridade.reverse()
    
    for j in similaridade:
        print(j)

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [46]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

standar = StandardScaler()
standar.fit(X)
X = standar.fit_transform(X)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

In [48]:
algorithms = []

algorithms.append(['DecisionTreeClassifier', DecisionTreeClassifier(random_state=0)])
algorithms.append(['RandomForestClassifier', RandomForestClassifier(n_estimators=10, random_state=0)])
algorithms.append(['GaussianNB', GaussianNB()])
algorithms.append(['RidgeClassifier', RidgeClassifier()])
algorithms.append(['MLPClassifier', MLPClassifier(random_state=True, max_iter=1000, learning_rate_init=0.0001, hidden_layer_sizes=(89))])
algorithms.append(['GradientBoostingClassifier', GradientBoostingClassifier()])
algorithms.append(['SVC', SVC(gamma='auto')])
algorithms.append(['KNeighborsClassifier', KNeighborsClassifier()])
algorithms.append(['LogisticRegression', LogisticRegression()])

fitAll(X_train, X_test, y_train, y_test, algorithms)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(0.5023041474654378, 'RandomForestClassifier')
(0.49539170506912444, 'SVC')
(0.4769585253456221, 'MLPClassifier')
(0.47465437788018433, 'GaussianNB')
(0.4700460829493088, 'DecisionTreeClassifier')
(0.45622119815668205, 'KNeighborsClassifier')
(0.45622119815668205, 'GradientBoostingClassifier')
(0.4539170506912442, 'RidgeClassifier')
(0.45161290322580644, 'LogisticRegression')


test = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000)
y_pred = fit(X_train, X_test, y_train, y_test, test)

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

In [49]:
y_test[300:301]

462    Loss
Name: status, dtype: object

In [50]:
X_test[300]

array([0.2238806 , 0.13975155, 0.01694915, 0.0588    , 0.24      ,
       0.39003095, 0.41666667, 0.65355556, 0.01612903, 0.03305785,
       0.        , 0.09090909, 0.10289389, 0.04504505, 0.3571    ,
       0.13043478, 0.23077751, 0.24      , 0.46155503, 0.02255639,
       0.        , 0.        , 0.08235294, 0.04826733, 0.05333333,
       0.4444    , 0.13043478, 0.35552   , 0.06451613, 0.24441756,
       0.02531646, 0.04477612, 1.        , 0.2173913 , 0.23404255,
       0.01694915, 0.0455    , 0.11428571, 0.19392   , 0.36956522,
       0.89154263, 0.02      , 0.11827957, 0.        , 0.15714286,
       0.0483871 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.16666667, 0.5       ,
       0.12080537, 0.10679612, 0.0390625 , 0.25      , 0.32142857,
       0.48460047, 0.22222222, 0.3299967 , 0.02234637, 0.01456311,
       0.        , 0.28888889, 0.2206655 , 0.25      , 0.7241    ,
       0.19354839, 0.2413954 , 0.08333333, 0.08154101, 0.05925

In [51]:
#print('Positive/Positive Ok', conf_matrix[0][0])
#print('Positive/Negative Er', conf_matrix[0][1])
#print('Negative/Negative Ok', conf_matrix[1][0])
#print('Negative/Positive Ok', conf_matrix[1][1])

In [52]:
clf_importances = dict(zip(x_columns, clf_ran.feature_importances_))

NameError: name 'clf_ran' is not defined

In [ ]:
clf_ran.feature_importances_[0:10], len(clf_ran.feature_importances_)

index = np.arange(len(x_columns[0:64]))
plt.bar(index, clf_ran.feature_importances_[0:64])
plt.xticks(index, x_columns, rotation='vertical')
plt.title('clf_ran')
plt.show()

index = np.arange(len(x_columns[64:128]))
plt.bar(index, clf_ran.feature_importances_[64:128])
plt.xticks(index, x_columns, rotation='vertical')
plt.title('clf_ran')
plt.show()